# Emotion Recognition in Greek Speech Using Wav2Vec 2.0

**Wav2Vec 2.0** is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2`s ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

During fine-tuning week hosted by HuggingFace, more than 300 people participated in tuning XLSR-Wav2Vec2's pretrained on low-resources ASR dataset for more than 50 languages. This model is fine-tuned using [Connectionist Temporal Classification](https://distill.pub/2017/ctc/) (CTC), an algorithm used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. Follow this [notebook](https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb#scrollTo=Gx9OdDYrCtQ1) for more information about XLSR-Wav2Vec2 fine-tuning.

This model was shown significant results in many low-resources languages. You can see the [competition board](https://paperswithcode.com/dataset/common-voice) or even testing the models from the [HuggingFace hub](https://huggingface.co/models?filter=xlsr-fine-tuning-week). 


In this notebook, we will go through how to use this model to recognize the emotional aspects of speech in a language (or even as a general view using for every classification problem). Before going any further, we need to install some handy packages and define some enviroment values.

In [1]:
%%capture

# !pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install jiwer
!pip install torchaudio
!pip install librosa==0.8.1
!pip show pyarrow
!pip install gdown

# Monitor the training process
!pip install wandb

In [2]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

In [3]:
master_csvfile = "../input/podcastfillersfull/PodcastFillers/metadata/PodcastFillers.csv"
clip_folder = "../input/podcastfillersfull/PodcastFillers/audio/clip_wav_unpadded"

In [4]:
data = []
event_df = pd.read_csv(master_csvfile)
for i, event in tqdm(event_df.iterrows()):

    clip_subset = event["clip_split_subset"]
    label = event['label_consolidated_vocab']

    tar_folder = os.path.join(clip_folder, clip_subset)
    tar_filepath = os.path.join(tar_folder, event["clip_name"])

    if not os.path.exists(tar_filepath):
        continue
    
    data.append({
        "subset": clip_subset,
        "path": tar_filepath,
        "label": label
    })

85803it [00:56, 1507.07it/s]


In [5]:
df = pd.DataFrame(data)
df.head()

,subset,path,label
0,extra,../input/podcastfillersfull/PodcastFillers/aud...,None
1,train,../input/podcastfillersfull/PodcastFillers/aud...,Music
2,validation,../input/podcastfillersfull/PodcastFillers/aud...,Uh
3,test,../input/podcastfillersfull/PodcastFillers/aud...,Breath
4,train,../input/podcastfillersfull/PodcastFillers/aud...,Music


In [6]:
df.describe()

,subset,path,label
count,13662,13662,13662
unique,4,13662,7
top,train,../input/podcastfillersfull/PodcastFillers/aud...,Um
freq,10375,1,3651


In [7]:
df.iloc[0]['path']

'../input/podcastfillersfull/PodcastFillers/audio/clip_wav_unpadded/extra/00000.wav'

Let's explore how many labels are in the dataset with what distribution.

In [8]:
print("Labels: ", df["label"].unique())
print()
df.groupby("label").count()[["path"]]

Labels:  ['None' 'Music' 'Uh' 'Breath' 'Um' 'Words' 'Laughter']



,path
label,
Breath,953
Laughter,940
Music,949
None,1319
Uh,2912
Um,3651
Words,2938


Let's display some random sample of the dataset and run it a couple of times to get a feeling for the audio and the emotional label.

In [9]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

# idx = np.random.randint(0, len(df))
idx = 5
sample = df.iloc[idx]
path = sample["path"]
label = sample["label"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

# speech, sr = torchaudio.load(path)
# speech = speech[0].numpy().squeeze()
# speech = librosa.resample(np.asarray(speech), sr, 16_000)
# ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 5
      Label: Um



In [10]:
!mkdir content
!mkdir content/data

In [11]:
!ls

__notebook_source__.ipynb  content


In [12]:
save_path = "content/data"

# train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["emotion"])
train_data = []
test_data = []
valid_data = []
extra_data = []
# event_df = pd.read_csv(master_csvfile)
for i, event in tqdm(df.iterrows()):

    if event['subset'] == 'train':
        train_data.append({
            'path': event['path'],
            'label': event['label']
        })
        
    elif event['subset'] == 'test':
        test_data.append({
            'path': event['path'],
            'label': event['label']
        })
        
    elif event['subset'] == 'validation':
        valid_data.append({
            'path': event['path'],
            'label': event['label']
        })
        
    elif event['subset'] == 'extra':
        extra_data.append({
            'path': event['path'],
            'label': event['label']
        })
    else:
        print("bad subset:", event['subset'])


13662it [00:00, 16060.61it/s]


In [13]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
valid_df = pd.DataFrame(valid_data)
extra_df = pd.DataFrame(extra_data)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)
valid_df.to_csv(f"{save_path}/valid.csv", sep="\t", encoding="utf-8", index=False)
extra_df.to_csv(f"{save_path}/extra.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)
print(valid_df.shape)
print(extra_df.shape)

(10375, 2)
(1568, 2)
(400, 2)
(1319, 2)


In [14]:
print("Labels: ", df["label"].unique())
print()
train_df.groupby("label").count()[["path"]]

Labels:  ['None' 'Music' 'Uh' 'Breath' 'Um' 'Words' 'Laughter']



,path
label,
Breath,852
Laughter,828
Music,792
Uh,2396
Um,3011
Words,2496


In [15]:
test_df

,path,label
0,../input/podcastfillersfull/PodcastFillers/aud...,Breath
1,../input/podcastfillersfull/PodcastFillers/aud...,Words
2,../input/podcastfillersfull/PodcastFillers/aud...,Breath
3,../input/podcastfillersfull/PodcastFillers/aud...,Um
4,../input/podcastfillersfull/PodcastFillers/aud...,Um
...,...,...
1563,../input/podcastfillersfull/PodcastFillers/aud...,Um
1564,../input/podcastfillersfull/PodcastFillers/aud...,Words
1565,../input/podcastfillersfull/PodcastFillers/aud...,Uh
1566,../input/podcastfillersfull/PodcastFillers/aud...,Um


## Prepare Data for Training

In [16]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric


data_files = {
    "train": "content/data/train.csv", 
    "test": "content/data/test.csv", 
    "validation": "content/data/valid.csv",
    "extra": "content/data/extra.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
test_dataset = dataset["test"]
eval_dataset = dataset["validation"]
extra_dataset = dataset["extra"]


print(train_dataset)
print(test_dataset)
print(eval_dataset)
print(extra_dataset)

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2844d04970436f16/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

Dataset({
    features: ['path', 'label'],
    num_rows: 10375
})
Dataset({
    features: ['path', 'label'],
    num_rows: 1568
})
Dataset({
    features: ['path', 'label'],
    num_rows: 400
})
Dataset({
    features: ['path', 'label'],
    num_rows: 1319
})


In [17]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [18]:
# We need to specify the input and output column
input_column = "path"
output_column = "label"

In [19]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 6 classes: ['Breath', 'Laughter', 'Music', 'Uh', 'Um', 'Words']


In order to preprocess the audio into our classification model, we need to set up the relevant Wav2Vec2 assets regarding our language in this case `jonatasgrosman/wav2vec2-large-xlsr-53-russian` . To handle the context representations in any audio length we use a merge strategy plan (pooling mode) to concatenate that 3D representations into 2D representations.

There are three merge strategies `mean`, `sum`, and `max`. In this example, we achieved better results on the mean approach. In the following, we need to initiate the config and the feature extractor from the Dimitris model.

In [20]:
from transformers import AutoConfig, Wav2Vec2Processor

In [21]:
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
pooling_mode = "mean"

In [80]:
id2label_ = {i: label for i, label in enumerate(label_list)}
id2label_

{0: 'Breath', 1: 'Laughter', 2: 'Music', 3: 'Uh', 4: 'Um', 5: 'Words'}

In [81]:
label_list

['Breath', 'Laughter', 'Music', 'Uh', 'Um', 'Words']

In [22]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

In [23]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


# Preprocess Data

So far, we downloaded, loaded, and split the SER dataset into train and test sets. The instantiated our strategy configuration for using context representations in our classification problem SER. Now, we need to extract features from the audio path in context representation tensors and feed them into our classification model to determine the emotion in the speech.

Since the audio file is saved in the `.wav` format, it is easy to use **[Librosa](https://librosa.org/doc/latest/index.html)** or others, but we suppose that the format may be in the `.mp3` format in case of generality. We found that the **[Torchaudio](https://pytorch.org/audio/stable/index.html)** library works best for reading in `.mp3` data.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a **map(...)** function accordingly. Also, we need to handle the string labels into integers for our specific classification task in this case, the **single-label classification** you may want to use for your **regression** or even **multi-label classification**.

In [24]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [25]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4
)

#0:   0%|          | 0/26 [00:00<?, ?ba/s]

#1:   0%|          | 0/26 [00:00<?, ?ba/s]

#2:   0%|          | 0/26 [00:00<?, ?ba/s]

#3:   0%|          | 0/26 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

## Model

Before diving into the training part, we need to build our classification model based on the merge strategy. 

In [26]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [27]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
#         print("hidden_states.shape: ", hidden_states.shape)
        logits = self.classifier(hidden_states)
#         print("logits: ", logits)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [28]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [29]:
from transformers import Trainer

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

In [31]:
is_regression = False

In [32]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
#         print("preds:", preds)
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained XLSR-Wav2Vec2 checkpoint into our classification model with a pooling strategy.

In [33]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [34]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [35]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/wav2vec2-uh_um-recognition",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=200,
    eval_steps=20,
    logging_steps=20,
    learning_rate=1e-4,
    save_total_limit=1,
)

For future use we can create our training script, we do it in a simple way. You can add more on you own.

Now, all instances can be passed to Trainer and we are ready to start training!

In [36]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [37]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy
20,1.502100,1.165604,0.585000
40,1.365000,1.003593,0.590000
60,1.337000,1.331674,0.542500
80,1.305500,0.900754,0.740000
100,1.034300,0.757725,0.722500
120,1.090900,0.732453,0.747500
140,1.309300,0.868702,0.727500
160,1.052900,0.798538,0.755000
180,1.028300,0.701075,0.770000
200,1.035800,0.701215,0.737500


In [41]:
!pip install gdown

In [43]:
# https://drive.google.com/file/d/1WfnmYuP58qC_68dKIcYgqqbPc_Usm6yU/view?usp=share_link

In [44]:
!gdown --id 1WfnmYuP58qC_68dKIcYgqqbPc_Usm6yU

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1WfnmYuP58qC_68dKIcYgqqbPc_Usm6yU
To: /kaggle/working/um_uh_1.wav
100%|██████████████████████████████████████| 41.7k/41.7k [00:00<00:00, 4.08MB/s]


In [87]:
def speech_file_to_array_fn(file_path):
    speech_array, sampling_rate = torchaudio.load(file_path)
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)
    return speech_array


def predict(speech_array):
    features = processor(speech_array, sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True, return_attention_mask=True)
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)
    
    with torch.no_grad():
        logits = model(input_values, attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    return pred_ids

In [88]:
speech_arr = speech_file_to_array_fn("um_uh_1.wav")

In [90]:
predict(speech_arr)

array([3])

In [39]:
# try to disable wandb to run on kaggle automatically
# import os
# os.environ["WANDB_DISABLED"] = "true"

### Save and load model

In [40]:
# saving model
torch.save(model.state_dict(), 'final_clf.pt')

In [82]:
model_loaded = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [83]:
model_loaded.load_state_dict(torch.load('final_clf.pt'))

<All keys matched successfully>

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [ ]:
model_loaded.to(device)

In [ ]:
model_loaded.to('cpu')

In [85]:
torch.save(model_loaded.state_dict(), 'interjections_clf_cpu.pt')

In [45]:
#experiment

In [60]:
# saving model
torch.save(model.state_dict(), 'final_clf_test.pt')

In [61]:
model_loaded_test = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
).to("cuda")

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [ ]:
# model_loaded_test.classifier.out_proj.weight

In [ ]:
model_loaded_test.load_state_dict(torch.load('final_clf_test.pt'))

In [ ]:
# model_loaded_test.classifier.out_proj.weight

In [ ]:
# model.classifier.out_proj.weight

## Evaluation

In [62]:
import librosa
from sklearn.metrics import classification_report

In [63]:
test_dataset = load_dataset("csv", data_files={"test": "content/data/test.csv"}, delimiter="\t")["test"]
test_dataset

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['path', 'label'],
    num_rows: 1568
})

In [64]:
# model_loaded


In [65]:
# model_name_or_path = "facebook/wav2vec2-base-960h"
model_name_or_path = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
# config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
# model_name_or_path = "final_clf.pt"
# model_new_loaded = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [66]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True, return_attention_mask=True)

#     print(features)
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)
    
    with torch.no_grad():
        logits = model(input_values, attention_mask).logits #attention_mask

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [67]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

  0%|          | 0/1568 [00:00<?, ?ex/s]

In [68]:
test_dataset

Dataset({
    features: ['path', 'label', 'speech'],
    num_rows: 1568
})

In [69]:
result = test_dataset.map(predict, batched=True, batch_size=8)

  0%|          | 0/196 [00:00<?, ?ba/s]

In [70]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

['Breath', 'Laughter', 'Music', 'Uh', 'Um', 'Words']

In [71]:
y_true = [config.label2id[name] for name in result["label"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

[0, 5, 0, 4, 4]
[0, 5, 0, 4, 4]


In [72]:
print(classification_report(y_true, y_pred, target_names=label_names))

              precision    recall  f1-score   support

      Breath       0.67      0.64      0.65        84
    Laughter       0.75      0.62      0.68        74
       Music       0.94      0.73      0.82       138
          Uh       0.76      0.64      0.69       413
          Um       0.85      0.83      0.84       519
       Words       0.62      0.84      0.71       340

    accuracy                           0.75      1568
   macro avg       0.76      0.72      0.73      1568
weighted avg       0.77      0.75      0.76      1568



In [ ]:
# precision    recall  f1-score   support

#       Breath       0.79      0.89      0.84        84
#     Laughter       0.78      0.85      0.81        74
#        Music       0.95      0.87      0.91       138
#           Uh       0.92      0.80      0.85       413
#           Um       0.92      0.95      0.93       519
#        Words       0.80      0.88      0.84       340

#     accuracy                           0.88      1568
#    macro avg       0.86      0.87      0.86      1568
# weighted avg       0.88      0.88      0.88      1568


#  precision    recall  f1-score   support

#       Breath       0.65      0.62      0.63        84
#     Laughter       0.77      0.62      0.69        74
#        Music       0.92      0.76      0.83       138
#           Uh       0.74      0.64      0.69       413
#           Um       0.85      0.81      0.83       519
#        Words       0.59      0.81      0.68       340

#     accuracy                           0.74      1568
#    macro avg       0.75      0.71      0.73      1568
# weighted avg       0.76      0.74      0.74      1568

# Prediction

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


STYLES = """
<style>
div.display_data {
    margin: 0 auto;
    max-width: 500px;
}
table.xxx {
    margin: 50px !important;
    float: right !important;
    clear: both !important;
}
table.xxx td {
    min-width: 300px !important;
    text-align: center !important;
}
</style>
""".strip()

def prediction(df_row):
    path, emotion = df_row["path"], df_row["emotion"]
    df = pd.DataFrame([{"Emotion": emotion, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    ipd.display(ipd.HTML(STYLES + df.to_html(**setup) + "<br />"))
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
    ipd.display(ipd.Audio(data=np.asarray(speech), autoplay=True, rate=sampling_rate))

    outputs = predict(path, sampling_rate)
    r = pd.DataFrame(outputs)
    ipd.display(ipd.HTML(STYLES + r.to_html(**setup) + "<br />"))

In [ ]:
test = pd.read_csv("/content/data/test.csv", sep="\t")
test.head()

In [ ]:
prediction(test.iloc[0])

In [ ]:
prediction(test.iloc[1])

In [ ]:
prediction(test.iloc[2])